In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.datasets import FashionMNIST
import torchvision.transforms as transforms
from tqdm import tqdm
from torch.utils.data import DataLoader, SequentialSampler

import matplotlib.pyplot as plt
import os

import numpy as np

import sys
sys.path.append("..")
from src.image_data import ImageClassificationDataset
from src.quantize import cluster_feat, KMeans
from src.ubmnist import UnbalanceFashionMNIST

In [2]:
DATASET = "ub_fmnist"
DATA_PATH = f'/mnt/ssd/ronak/datasets/{DATASET}'
root = DATA_PATH
MODEL_NAME = "convmnist_e24"

### Download and View Data

In [ ]:
data_dir = '/mnt/hdd2/liu16/data'
val_size = 1000
smooth = 0.005
size = 14400

trainset = UnbalanceFashionMNIST(
    root=data_dir, train=True, val_size=val_size,
    download=True, transform=transforms.ToTensor(),
    smooth=smooth, size=size)
mean = (trainset.data.float().mean().item() / 255,)
std = (trainset.data.float().std().item() / 255,)

print(mean)
print(std)

transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean, std),])

# load again using transforms
train_data = UnbalanceFashionMNIST(root=data_dir, train=True, val_size=val_size,
            download=True, transform=transform, smooth=smooth, size=size)
# train_data = UnbalanceFashionMNIST(
#     root=data_dir, train=True, val_size=val_size,
#     download=True, transform=transforms.ToTensor(),
#     smooth=smooth, size=size)
test_data = FashionMNIST(root, download=True, train=False)

x_train = torch.clone(train_data.data).numpy()
y_train = torch.clone(train_data.targets).numpy()
x_test = torch.clone(test_data.data).numpy()
y_test = torch.clone(test_data.targets).numpy()

print(x_train.shape)
print(y_train.shape)

In [ ]:
new_mean = mean[0] * 255
new_std = std[0] * 255

x_train.mean() / 255

In [ ]:
x = train_data.data
y = np.array(train_data.targets)

print(x.shape)
print(y.shape)

In [ ]:
labels, counts = np.unique(y, return_counts=True)
print(counts / counts.sum())

In [ ]:
# tiling is unnecessary as we will use a grayscale quantization model

# x_train = np.tile(train_data.data[..., None], 3) / 255
x_train = (x_train - new_mean) / new_std * 255
y_train = train_data.targets
# x_test =  np.tile(test_data.data[..., None], 3) / 255
x_test =  (test_data.data - new_mean) / new_std * 255
y_test = test_data.targets

print(x_train[0, :, :])
print(x_train.shape)
print(y_train.shape)
print(len(np.unique(y_train)))

np.save(os.path.join(root, "x_train.npy"), x_train)
np.save(os.path.join(root, "y_train.npy"), y_train)
np.save(os.path.join(root, "x_test.npy"), x_test)
np.save(os.path.join(root, "y_test.npy"), y_test)

In [ ]:
image = x_train[0]

fig, ax = plt.subplots(figsize=(2, 2))
ax.axis("off")
ax.imshow(image)
plt.show()

### Load Embedding Model

In [3]:
DEVICE = 'cuda:3'

In [4]:
class ConvMnist(nn.Module):
    def __init__(self, hidden_dim=512):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5, padding=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5, padding=2)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=5, padding=2)
        self.fc1 = nn.Linear(64*3*3, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, 10)

    def forward(self, x, return_feats=False):
        x = F.max_pool2d(F.relu(self.conv1(x)), 2)  # conv1
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)  # conv2
        x = F.max_pool2d(F.relu(self.conv3(x)), 2) # conv3 
        x = x.view(x.shape[0], -1) # flatten
        features = self.fc1(x)
        x = F.relu(features)
        x = self.fc2(x)
        if return_feats:
            return x, features
        else:
            return x

In [5]:
model_path = '/mnt/hdd2/liu16/convnet/unbalance_fashion_mnist_smooth0.005_size14400_v1000_b256/checkpoints/epoch_24.pt'
num_hidden = 512

record = torch.load(model_path)
model = model = ConvMnist(num_hidden)
model.load_state_dict(record['state_dict'])
model.to(DEVICE)

ConvMnist(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv3): Conv2d(64, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (fc1): Linear(in_features=576, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=10, bias=True)
)

In [6]:
root = DATA_PATH

x_train = np.expand_dims(np.load(os.path.join(root, "x_train.npy")), axis=1) / 255
print(x_train[0, 0, :10, :10])
y_train = np.load(os.path.join(root, "y_train.npy"))
print(x_train.shape)

batch_size = 256
# transform = transforms.Compose([
#             # transforms.ToTensor(),
#             transforms.Normalize(mean, std),])
# train_dataset = ImageClassificationDataset(x_train, y_train, transform=transform)
train_dataset = ImageClassificationDataset(x_train, y_train)
dataloader = DataLoader(
    train_dataset, sampler=SequentialSampler(train_dataset), batch_size=batch_size
)

[[-0.80041401 -0.80041401 -0.80041401 -0.80041401 -0.80041401 -0.80041401
  -0.80041401 -0.80041401 -0.80041401 -0.80041401]
 [-0.80041401 -0.80041401 -0.80041401 -0.80041401 -0.80041401 -0.80041401
  -0.80041401 -0.78959127 -0.80041401  0.51996042]
 [-0.80041401 -0.80041401 -0.80041401 -0.80041401 -0.80041401 -0.80041401
  -0.80041401 -0.80041401 -0.80041401  1.48318438]
 [-0.80041401 -0.80041401 -0.80041401 -0.80041401 -0.80041401 -0.77876853
  -0.80041401 -0.49737726  1.42907068  1.39660245]
 [-0.80041401 -0.80041401 -0.80041401 -0.80041401 -0.75712305 -0.80041401
  -0.63807289  1.37495697  1.34248875  1.15850215]
 [-0.80041401 -0.80041401 -0.80041401 -0.80041401 -0.80041401 -0.80041401
   1.08274296  1.39660245  1.12603392  1.14767941]
 [-0.80041401 -0.80041401 -0.80041401 -0.80041401 -0.80041401 -0.80041401
   1.03945199  1.29919778  1.19097037  1.14767941]
 [-0.80041401 -0.80041401 -0.80041401 -0.80041401 -0.80041401 -0.70300934
   1.31002052  1.24508408  1.19097037  1.20179311]


In [7]:
all_image_features, all_labels, all_idx = [], [], []
with torch.no_grad():
    for i, batch in tqdm(enumerate(dataloader)):
        idx, images, labels = batch
        image_features = model(images.to(DEVICE), return_feats=True)[1].squeeze()
        all_image_features.append(image_features)
        all_labels.append(labels)
        all_idx.append(idx)
        
all_image_features = torch.cat(all_image_features).cpu().detach().numpy()
all_labels = torch.cat(all_labels).cpu().detach().numpy()
all_idx = torch.cat(all_idx).cpu().detach().numpy()

57it [00:00, 136.11it/s]


In [30]:
torch.save(all_image_features, os.path.join(DATA_PATH, f"{MODEL_NAME}_features.pt"))
torch.save(all_labels, os.path.join(DATA_PATH, f"{MODEL_NAME}_labels.pt"))
torch.save(all_idx, os.path.join(DATA_PATH, f"{MODEL_NAME}_idx.pt"))

## Perform Quantization

In [4]:
NUM_CLUSTERS = 100
SEED = 20220711

In [5]:
all_image_features = torch.load(os.path.join(DATA_PATH, f"{MODEL_NAME}_features.pt"))
all_labels = torch.load(os.path.join(DATA_PATH, f"{MODEL_NAME}_labels.pt"))
all_idx = torch.load(os.path.join(DATA_PATH, f"{MODEL_NAME}_idx.pt"))

In [6]:
image_labels, image_cluster = cluster_feat(all_image_features, NUM_CLUSTERS, seed=SEED)

label_to_idx = np.argsort(all_idx)
print(all_idx[label_to_idx])

# have the labels correspond to the indices in order.
image_labels_sorted = image_labels[label_to_idx]
class_labels_sorted = all_labels[label_to_idx]

print(image_labels_sorted.shape)
print(class_labels_sorted.shape)

[    0     1     2 ... 14391 14392 14393]
(14394,)
(14394,)


In [7]:
model_name = "convmnist_e24_train"
save_dir = f'/mnt/ssd/ronak/datasets/{DATASET}/quantization/{model_name}_kmeans_{NUM_CLUSTERS}'

os.makedirs(save_dir, exist_ok=True)

np.save(os.path.join(save_dir, f'image_labels.npy'), image_labels_sorted)
np.save(os.path.join(save_dir, f'class_labels.npy'), class_labels_sorted)

_, counts = np.unique(all_labels, return_counts=True)
y_marginal = counts/np.sum(counts)
x_marginal = image_cluster.marginal

np.save(os.path.join(save_dir, f'image_marginal.npy'), x_marginal)
np.save(os.path.join(save_dir, f'class_marginal.npy'), y_marginal)

In [8]:
len(x_marginal)

98

### Alternate Quantization

In [5]:
import pickle

In [6]:
all_image_features = torch.load(os.path.join(DATA_PATH, f"{MODEL_NAME}_features.pt"))
all_labels = torch.load(os.path.join(DATA_PATH, f"{MODEL_NAME}_labels.pt"))
all_idx = torch.load(os.path.join(DATA_PATH, f"{MODEL_NAME}_idx.pt"))

In [7]:
smooth = 0.005
size = 14400
epoch = 24
val_size = 1000
batch_size = 256
NUM_CLUSTERS = 100

model_name = f'unbalance_fashion_mnist_smooth{smooth}_size{size}_v{val_size}_b{batch_size}'
CKP_PATH = f'/mnt/hdd2/liu16/convnet'

# cluster_path = f'{CKP_PATH}/{model_name}/kmeans_c{NUM_CLUSTERS}_v{val_size}_e{epoch}_image.pkl'  # remove _v{val_size}
cluster_path = f'kmeans_c{NUM_CLUSTERS}_v{val_size}_e{epoch}_image.pkl'  # remove _v{val_size}
cluster_path
with open(cluster_path, 'rb+') as f:
    image_cluster = pickle.load(f)

/home/ronak/miniconda3/envs/dl/lib/python3.11/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator PCA from version 0.24.2 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [8]:
type(image_cluster)

src.quantize.KMeans

In [9]:
image_labels = image_cluster.clustering(all_image_features)

In [10]:
len(np.unique(image_labels))

91

In [11]:
len(image_cluster.marginal)

92

In [12]:
label_to_idx = np.argsort(all_idx)
print(all_idx[label_to_idx])

# have the labels correspond to the indices in order.
image_labels_sorted = image_labels[label_to_idx]
class_labels_sorted = all_labels[label_to_idx]

print(image_labels_sorted.shape)
print(class_labels_sorted.shape)

[    0     1     2 ... 14391 14392 14393]
(14394,)
(14394,)


In [13]:
save_dir = f'/mnt/ssd/ronak/datasets/{DATASET}/quantization/{MODEL_NAME}_kmeans_{NUM_CLUSTERS}'

os.makedirs(save_dir, exist_ok=True)

np.save(os.path.join(save_dir, f'image_labels.npy'), image_labels_sorted)
np.save(os.path.join(save_dir, f'class_labels.npy'), class_labels_sorted)

_, counts = np.unique(all_labels, return_counts=True)
y_marginal = counts/np.sum(counts)
x_marginal = image_cluster.marginal

np.save(os.path.join(save_dir, f'image_marginal.npy'), x_marginal)
np.save(os.path.join(save_dir, f'class_marginal.npy'), y_marginal)

In [14]:
save_dir

'/mnt/ssd/ronak/datasets/ub_fmnist/quantization/convmnist_e24_kmeans_100'

### ConvNext Quantization

In [3]:
from torchvision.models import convnext_base, ConvNeXt_Base_Weights
from torchvision.models.feature_extraction import get_graph_node_names, create_feature_extractor
from torch.utils.data import DataLoader, SequentialSampler

In [4]:
DATASET = "ub_fmnist"
DATA_PATH = f'/mnt/ssd/ronak/datasets/{DATASET}'
root = DATA_PATH
DEVICE = "cuda:2"
MODEL_NAME = "convnext"

In [5]:
model = convnext_base(weights=ConvNeXt_Base_Weights.IMAGENET1K_V1).to(DEVICE)
# train_nodes, eval_nodes = get_graph_node_names(model) # use to check layer names
return_nodes = {
    # node_name: user-specified key for output dict
    'avgpool': 'features',
}
body = create_feature_extractor(model, return_nodes=return_nodes)

/home/ronak/miniconda3/envs/dl/lib/python3.11/site-packages/torch/overrides.py:110: UserWarning: 'has_cuda' is deprecated, please use 'torch.backends.cuda.is_built()'
  torch.has_cuda,
/home/ronak/miniconda3/envs/dl/lib/python3.11/site-packages/torch/overrides.py:111: UserWarning: 'has_cudnn' is deprecated, please use 'torch.backends.cudnn.is_available()'
  torch.has_cudnn,
/home/ronak/miniconda3/envs/dl/lib/python3.11/site-packages/torch/overrides.py:117: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  torch.has_mps,
/home/ronak/miniconda3/envs/dl/lib/python3.11/site-packages/torch/overrides.py:118: UserWarning: 'has_mkldnn' is deprecated, please use 'torch.backends.mkldnn.is_available()'
  torch.has_mkldnn,


In [6]:
data_dir = '/mnt/hdd2/liu16/data'
val_size = 1000
smooth = 0.005
size = 14400

trainset = UnbalanceFashionMNIST(
    root=data_dir, train=True, val_size=val_size,
    download=True, transform=transforms.ToTensor(),
    smooth=smooth, size=size)
mean = trainset.data.float().mean().item()
std = trainset.data.float().std().item()

print(mean)
print(std)

73.95668029785156
92.39803314208984


/home/ronak/resnets/notebooks/../src/ubmnist.py:295: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at ../torch/csrc/utils/tensor_numpy.cpp:206.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [8]:
root = DATA_PATH
# x_train = np.transpose(np.load(os.path.join(root, "x_train.npy")), axes=[0, 3, 1, 2])
x_train = std * np.load(os.path.join(root, "x_train.npy")) / 255 + mean
x_train = np.transpose(np.tile(x_train[..., None], 3), axes=[0, 3, 1, 2])
y_train = np.load(os.path.join(root, "y_train.npy"))
print(x_train.shape)
print(y_train.shape)
print(x_train[0, 0])

batch_size = 256
transforms_ = ConvNeXt_Base_Weights.IMAGENET1K_V1.transforms()
train_dataset = ImageClassificationDataset(x_train, y_train, transforms_)
dataloader = DataLoader(
    train_dataset, sampler=SequentialSampler(train_dataset), batch_size=batch_size
)

(14394, 3, 28, 28)
(14394,)
[[  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  15.  70.  92. 101.
  114.  93.  79.  74.  25.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   1.   0. 122. 236. 231. 237. 237.
  240. 235. 232. 227. 226.  67.   0.   0.   0.   1.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0. 211. 234. 206. 190. 220.
  229. 231. 207. 190. 211. 206.   0.   0.   1.   1.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   2.   0.  28. 206. 203. 209. 208. 180. 215.
  225. 223. 153. 207. 222. 209. 155.   0.   0.   1.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   4.   0.  15. 201. 198. 181. 174. 191. 196. 196.
  245. 200. 202. 220. 191. 177. 199. 173.   0.   0.   2.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0. 174. 203. 178. 180. 181. 180. 172. 173.
  225. 190. 181. 172. 171. 182. 172. 203. 141.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0. 170. 194. 184. 180. 175. 179. 173. 172.
  235. 187. 171. 179. 175. 180

In [9]:
all_image_features, all_labels, all_idx = [], [], []
with torch.no_grad():
    for i, batch in tqdm(enumerate(dataloader)):
        idx, images, labels = batch
        image_features = body(images.to(DEVICE))['features'].squeeze()
        image_features /= image_features.norm(dim=-1, keepdim=True)
        all_image_features.append(image_features)
        all_labels.append(labels)
        all_idx.append(idx)
        
all_image_features = torch.cat(all_image_features).cpu().detach().numpy()
all_labels = torch.cat(all_labels).cpu().detach().numpy()
all_idx = torch.cat(all_idx).cpu().detach().numpy()

torch.save(all_image_features, os.path.join(DATA_PATH, "convnext_base_features.pt"))
torch.save(all_labels, os.path.join(DATA_PATH, "convnext_base_labels.pt"))
torch.save(all_idx, os.path.join(DATA_PATH, "convnext_base_idx.pt"))

0it [00:00, ?it/s]/home/ronak/miniconda3/envs/dl/lib/python3.11/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
57it [01:56,  2.05s/it]


In [10]:
NUM_CLUSTERS = 100
SEED = 20220711

image_labels, image_cluster = cluster_feat(all_image_features, NUM_CLUSTERS, seed=SEED)

label_to_idx = np.argsort(all_idx)
print(all_idx[label_to_idx])

# have the labels correspond to the indices in order.
image_labels_sorted = image_labels[label_to_idx]
class_labels_sorted = all_labels[label_to_idx]

print(image_labels_sorted.shape)
print(class_labels_sorted.shape)

model_name = "convnext_base"
save_dir = f'/mnt/ssd/ronak/datasets/{DATASET}/quantization/{model_name}_kmeans_{NUM_CLUSTERS}'

os.makedirs(save_dir, exist_ok=True)

np.save(os.path.join(save_dir, f'image_labels.npy'), image_labels_sorted)
np.save(os.path.join(save_dir, f'class_labels.npy'), class_labels_sorted)

_, counts = np.unique(all_labels, return_counts=True)
y_marginal = counts/np.sum(counts)
x_marginal = image_cluster.marginal

np.save(os.path.join(save_dir, f'image_marginal.npy'), x_marginal)
np.save(os.path.join(save_dir, f'class_marginal.npy'), y_marginal)

print(save_dir)

[    0     1     2 ... 14391 14392 14393]
(14394,)
(14394,)
/mnt/ssd/ronak/datasets/ub_fmnist/quantization/convnext_base_kmeans_100
